<a href="https://colab.research.google.com/github/RuyiSheep/closet_app/blob/main/closet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!nvidia-smi -L || echo "Running on CPU"
!pip -q install timm==1.0.9 torchmetrics==1.4.0.post0 pandas==2.2.2 scikit-learn==1.5.1 albumentations==1.4.14 opencv-python==4.10.0.84

import os, random, json, math, time
import numpy as np
import pandas as pd
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.preprocessing import LabelEncoder
from typing import Dict, Any, List, Tuple

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as T
import timm
from torchmetrics.classification import MulticlassAccuracy

# Reproducibility
seed = 42
random.seed(seed); np.random.seed(seed); torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
DEVICE


/bin/bash: line 1: nvidia-smi: command not found
Running on CPU
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.4/42.4 kB 698.0 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 23.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 868.8/868.8 kB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 57.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.0/178.0 kB 6.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.5/62.5 MB 11.4 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
umap-learn 0.5.9.post2 requires scikit-learn>=1.6, but you have scikit-learn 1.5.1 which is incompatible.


'cpu'


# STEP 1 — Data Preparation


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls "/content/drive/MyDrive/Colab Notebooks/closet_app/annotations/"

closet_annotations.csv	closet_annotations.xltx


In [ ]:
# read annotation data and image data
CSV_PATH  = "/content/drive/MyDrive/Colab Notebooks/closet_app/annotations/closet_annotations.csv"
df = pd.read_csv(CSV_PATH)

OLD_PREFIX = "/Users/ruyiyang/Desktop/closet_app/closet/"
NEW_PREFIX = "/content/drive/MyDrive/Colab Notebooks/closet_app/closet/"

df[PATH_COL] = df[PATH_COL].str.replace(OLD_PREFIX, NEW_PREFIX, regex=False)

import ast

def pick_first(c):
    if isinstance(c, str):
        try:
            # if stored like "['red','blue']"
            vals = ast.literal_eval(c)
            if isinstance(vals, list) and len(vals) > 0:
                return vals[0]
        except:
            # if stored like "red, blue"
            return c.split(",")[0].strip()
    return "unknown"

df["color_single"] = df["colors"].apply(pick_first)

In [ ]:
display(df.head())

,item_id,path,category,colors,warmth,formality,color_single,category_id,color_id
0,1,/content/drive/MyDrive/Colab Notebooks/closet_...,t-shirt,['white'],1,1,white,16,9
1,2,/content/drive/MyDrive/Colab Notebooks/closet_...,t-shirt,"[""grey""]",2,2,grey,16,4
2,3,/content/drive/MyDrive/Colab Notebooks/closet_...,t-shirt,"[""black"", ""white""]",2,2,black,16,0
3,4,/content/drive/MyDrive/Colab Notebooks/closet_...,t-shirt,"[""grey""]",2,3,grey,16,4
4,5,/content/drive/MyDrive/Colab Notebooks/closet_...,hoodie,"[""white""]",3,2,white,6,9


In [ ]:

CSV_PATH  = "/content/drive/MyDrive/Colab Notebooks/closet_app/annotations/closet_annotations.csv"
IMG_ROOT  = "/content/drive/MyDrive/Colab Notebooks/closet_app/closet/"
ID_COL    = "path"

# Label columns (rename to match your CSV)
CATEGORY_COL  = "category"   # e.g., tops/bottoms/outer/shoes
COLOR_COL     = "color_single"      # e.g., black/white/blue/...
WARMTH_COL    = "warmth"     # integer 1..5
FORMALITY_COL = "formality"  # integer 1..5


# encode string label

cat_encoder = LabelEncoder()
col_encoder = LabelEncoder()

df["category_id"] = cat_encoder.fit_transform(df[CATEGORY_COL].astype(str))
df["color_id"]    = col_encoder.fit_transform(df[COLOR_COL].astype(str))
df["warmth_id"]   = df[WARMTH_COL] - 1
df["formality_id"] = df[FORMALITY_COL] - 1



In [ ]:
def process_image(img_path):
    """Loads, resizes, and preprocesses an image for ResNet50."""
    try:
        # Load the image, ensuring the target size is 224x224
        img = image.load_img(img_path, target_size=(224, 224))

        # Convert the image to a numpy array
        img_array = image.img_to_array(img)

        # Expand dimensions to create a "batch" of 1
        expanded_img_array = np.expand_dims(img_array, axis=0)

        # Preprocess the image for the ResNet50 model
        return resnet50.preprocess_input(expanded_img_array)
    except Exception as e:
        print(f"Error processing image {img_path}: {e}")
        return None

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications import resnet50
from tensorflow.keras.preprocessing import image
from tqdm import tqdm
# Use tqdm to create a smart progress bar for our loop
tqdm.pandas()

model = resnet50.ResNet50(weights='imagenet', include_top=False, pooling='avg', input_shape=(224, 224, 3))


# Apply the function to the 'path' column to get preprocessed images
# Then, use the model to predict (extract features) for all of them.
# Note: This may be slow if you have many images.
def extract_features(path):
    preprocessed_image = process_image(path)
    if preprocessed_image is not None:
        features = model.predict(preprocessed_image, verbose=0)
        return features.flatten()
    return None

print("Starting feature extraction... This might take a while.")
df['image_vector'] = df['path'].progress_apply(extract_features)
print("Feature extraction complete!")

Starting feature extraction... This might take a while.


100%|██████████| 91/91 [00:41<00:00,  2.21it/s]

Feature extraction complete!


In [ ]:

# Save the new DataFrame with features to a new file so you don't have to do this again!
output_path = '/content/drive/My Drive/Colab Notebooks/closet_app/annotations/annotations_with_features.csv'
df.to_csv(output_path, index=False)

print(f"\nSuccessfully saved the new data to {output_path}")


Successfully saved the new data to /content/drive/My Drive/Colab Notebooks/closet_app/annotations/annotations_with_features.csv


# STEP 2 Develop logic

In [ ]:
# load data
df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/closet_app/annotations/annotations_with_features.csv')
df['image_vector'] = df['image_vector'].apply(
    lambda x: np.fromstring(x.strip('[]'), sep=' ')
)

In [ ]:
df.head()

,item_id,path,category,colors,warmth,formality,color_single,category_id,color_id,warmth_id,formality_id,image_vector
0,1,/content/drive/MyDrive/Colab Notebooks/closet_...,t-shirt,['white'],1,1,white,16,9,0,0,[0.57026917 3.8535056 0.2709365 ... 0.032181...
1,2,/content/drive/MyDrive/Colab Notebooks/closet_...,t-shirt,"[""grey""]",2,2,grey,16,4,1,1,[0.01035272 1.9069592 0.05552468 ... 0.049274...
2,3,/content/drive/MyDrive/Colab Notebooks/closet_...,t-shirt,"[""black"", ""white""]",2,2,black,16,0,1,1,[0. 1.7564937 0. ... 0. ...
3,4,/content/drive/MyDrive/Colab Notebooks/closet_...,t-shirt,"[""grey""]",2,3,grey,16,4,1,2,[0. 1.3682871 0.0021872 ... 0.007906...
4,5,/content/drive/MyDrive/Colab Notebooks/closet_...,hoodie,"[""white""]",3,2,white,6,9,2,1,[0.6764801 1.4756545 0.09172161 ... 0.153257...


/tmp/ipython-input-48785574.py:2: DeprecationWarning: string or file could not be read to its end due to unmatched data; this will raise a ValueError in the future.
  lambda x: np.fromstring(x.strip('[]'), sep=' ')


In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
import ast # Import the ast module

def suggest_outfit_v2(closet_df, weather_input, formality_input):
    """
    Suggests a visually coordinated outfit using image_vectors.
    """

    # 1. Map weather to warmth (same as before)
    weather_map = {'hot': 0, 'tepid': 1, 'warm': 2, 'cold': 3,  'freezing': 4} # Added tepid and freezing to map
    required_warmth = weather_map.get(weather_input.lower())
    if required_warmth is None:
        return "Invalid weather input."

    # Define categories (same as before)
    top_categories = ['t-shirt', 'sweater', 'shirt']
    bottom_categories = ['jeans', 'skirt', 'shorts']
    outerwear_categories = ['coat', 'jacket', 'blazer']
    shoes_categories = ['sneakers', 'slipper', 'flat', 'boot', 'heel']


    outfit = {}

    # 2. Select a Base Item (the top)
    possible_tops = closet_df[
        (closet_df['category'].isin(top_categories)) &
        (closet_df['warmth_id'] <= required_warmth) &
        (closet_df['formality_id'] >= formality_input) # Changed to >= for flexibility
    ].copy() # Add .copy() to avoid SettingWithCopyWarning


    if possible_tops.empty:
        return "Couldn't find a suitable top."

    # Randomly select one top to build the outfit around
    selected_top = possible_tops.sample(1)
    outfit['top'] = selected_top.to_dict('records')[0]

    # Get the vector for the selected top
    # Ensure the image_vector is a numpy array before reshaping
    top_vector = np.array(outfit['top']['image_vector']).reshape(1, -1)


    # 3. Find the Best Matching Bottom
    possible_bottoms = closet_df[
        (closet_df['category'].isin(bottom_categories)) &
        (closet_df['formality_id'] >= formality_input)
    ].copy() # Add .copy()


    if possible_bottoms.empty:
        return "Found a top, but no suitable bottoms."

    # Calculate similarity between the top and ALL possible bottoms
    # Ensure bottom_vectors are numpy arrays
    bottom_vectors = np.vstack(possible_bottoms['image_vector'].values)
    similarities = cosine_similarity(top_vector, bottom_vectors)

    # Find the index of the BEST matching bottom
    best_match_index = np.argmax(similarities)

    # Select the best bottom from the filtered dataframe
    selected_bottom = possible_bottoms.iloc[[best_match_index]]
    outfit['bottom'] = selected_bottom.to_dict('records')[0]

    # 4. Add shoes
    possible_shoes = closet_df[
        (closet_df['category'].isin(shoes_categories)) &
        (closet_df['formality_id'] >= formality_input) &
        (closet_df['warmth_id'] <= required_warmth) # Added warmth filter for shoes
    ].copy() # Add .copy()

    if not possible_shoes.empty:
        # For simplicity, let's pick a random shoe for now
        outfit['shoes'] = possible_shoes.sample(1).to_dict('records')[0]

    # 5. Add outerwear if cold
    if weather_input.lower() in ['cold', 'freezing']: # Added freezing
         possible_outerwear = closet_df[
            (closet_df['category'].isin(outerwear_categories)) &
            (closet_df['formality_id'] >= formality_input)
        ].copy() # Add .copy()

         if not possible_outerwear.empty:
            # Pick a random outerwear piece
            outfit['outerwear'] = possible_outerwear.sample(1).to_dict('records')[0]


    return outfit


# --- Example Usage ---
print("--- Smart Suggestion for a hot, casual day ---")
smart_outfit = suggest_outfit_v2(df, weather_input='hot', formality_input=0)

if isinstance(smart_outfit, dict):
    # This outfit is no longer random, it's visually matched! 🎨
    top = smart_outfit.get('top')
    bottom = smart_outfit.get('bottom')
    shoes = smart_outfit.get('shoes')
    outerwear = smart_outfit.get('outerwear')

    if top:
        print(f"Visually Matched Top: {top['path']} (Item ID: {top['item_id']})")
    if bottom:
        print(f"Visually Matched Bottom: {bottom['path']} (Item ID: {bottom['item_id']})")
    if shoes:
        print(f"Suggested Shoes: {shoes['path']} (Item ID: {shoes['item_id']})")
    if outerwear:
         print(f"Suggested Outerwear: {outerwear['path']} (Item ID: {outerwear['item_id']})")
else:
    print(smart_outfit)

print("\n" + "="*40 + "\n")

print("--- Smart Suggestion for a cold, formal event ---")
smart_outfit_formal = suggest_outfit_v2(df, weather_input='cold', formality_input=3)

if isinstance(smart_outfit_formal, dict):
    top = smart_outfit_formal.get('top')
    bottom = smart_outfit_formal.get('bottom')
    shoes = smart_outfit_formal.get('shoes')
    outerwear = smart_outfit_formal.get('outerwear')

    if top:
        print(f"Visually Matched Top: {top['path']} (Item ID: {top['item_id']})")
    if bottom:
        print(f"Visually Matched Bottom: {bottom['path']} (Item ID: {bottom['item_id']})")
    if shoes:
        print(f"Suggested Shoes: {shoes['path']} (Item ID: {shoes['item_id']})")
    if outerwear:
         print(f"Suggested Outerwear: {outerwear['path']} (Item ID: {outerwear['item_id']})")
else:
    print(smart_outfit_formal)

--- Smart Suggestion for a hot, casual day ---
Visually Matched Top: /content/drive/MyDrive/Colab Notebooks/closet_app/closet/top/tshirt-white.jpg (Item ID: 1)
Visually Matched Bottom: /content/drive/MyDrive/Colab Notebooks/closet_app/closet/bottom/bot-4.avif (Item ID: 50)
Suggested Shoes: /content/drive/MyDrive/Colab Notebooks/closet_app/closet/shoe/shoe-13.avif (Item ID: 88)


--- Smart Suggestion for a cold, formal event ---
Found a top, but no suitable bottoms.
